### 2. Passing Leaders

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math
from datetime import date
import re

In [2]:
# CFB lines url
espn_url = 'https://www.espn.co.uk/college-football/stats/player/_/view/offense/table/passing/sort/passingYards/dir/desc'

In [3]:
# get raw data
page = requests.get(espn_url)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="fittPageContainer") # found by inspecting html

In [5]:
raw_output = [i.text for i in results.find_all("td", class_="Table__TD")]

In [7]:
n_rows = 50

# get qb names
qbs = []
for i in range(n_rows*2):
    if i % 2 != 0:
        qbs.append(raw_output[i])

school = []
qb_clean = []

# split out the school from the surname of the QB
# could probably use REGEX to improve this, but it's fine
for qb in qbs:
    if qb[len(qb)-4].isupper():
        # 4 letter team abbrv
        qb_clean.append(qb[:len(qb)-4])
        school.append(qb[len(qb)-4:])
    elif qb[len(qb)-3].isupper():
        # 3 letter abbrv
        qb_clean.append(qb[:len(qb)-3])
        school.append(qb[len(qb)-3:])
    else:
        # 2 letter abbrv
        qb_clean.append(qb[:len(qb)-2])
        school.append(qb[len(qb)-2:])
        
# get other stats
raw_stats = raw_output[n_rows*2:]
table = pd.DataFrame(np.reshape(raw_stats, (int(len(raw_stats)/11), 11)), 
                     columns = ['POS','CMP','ATT','CMP%','YDS','AVG','LNG','TD','INT','SACK','RTG'])
table.insert(0, 'NAME', qb_clean)
table.insert(1, 'ABBRV', school)

In [8]:
table.head()

,NAME,ABBRV,POS,CMP,ATT,CMP%,YDS,AVG,LNG,TD,INT,SACK,RTG
0,Jake Haener,FRES,QB,164,236,69.5,"2,230",9.4,86,18,6,10,168.9
1,Brennan Armstrong,UVA,QB,145,224,64.7,"1,973",8.8,65,14,4,13,155.8
2,Will Rogers,MSST,QB,215,284,75.7,"1,862",6.6,41,14,2,10,145.6
3,Kenny Pickett,PITT,QB,121,168,72.0,"1,731",10.3,67,19,1,8,194.7
4,Bailey Zappe,WKU,QB,133,183,72.7,"1,712",9.4,73,16,2,6,177.9


In [24]:
# save out as dataset
todays_date = str(date.today())
table.to_csv('data/'+todays_date+'-qb-records.csv', index=False)